In [1]:
import numpy as np
import tensorflow as tf
import os
import random
from tensorflow.keras import Model # if only machine learning were this easy :P
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Dropout

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
#import any other libraries you want here:
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from skimage.io import imread, imshow, imsave

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('drive/My Drive/Extracurriculars/IEEE/PocketRacers/dataset.csv')
df = df.sample(frac=1, random_state = 0).reset_index(drop=True)

x = df['transformed_file_name'].values
y = df['class_indices'].values

new_x = []
for i in range(len(x)):
  im = imread(x[i])
  new_x.append(im)

x = new_x
x = np.array(x)
# Let's grab some validation data from our training dataset!
# 5000 datapoints should be enough.
# Remember to remove them from your training data!
x_val = x[:300] #any 5000 points from x_train
y_val = y[:300] #same 5000 points from y_train

x_train = x[300:] #x_train - x_val
y_train = y[300:] #y_train - y_val

# This stuff will throw an error until the above is completed.
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(1200, 480, 640, 3)
(1200,)
(300, 480, 640, 3)
(300,)


In [ ]:
input_shape = x_train[0].shape

# First, we declare the size of our inputs
inputs = tf.keras.Input(shape=input_shape)

# YOUR TURN:
# This is where our layers go. Add/modify the layers of your
# neural network here.
# Note the basic syntax for defining this network. Each layer
# is defined by calling a function on the layer before; the functions
# themselves and their parameters are outlined in the tf.keras docs

#You should customize and experiment with your architecture, but if you want
#a (somewhat arbitrary) place to start, try the following:
    # 5x5 convolutional layer with 10 output channels
    # maxpool 2
    # 3x3 convolution with 20 output channels
    # Batchnorm
    # Maxpool 2
    # Flatten
    # Dropout = 0.5
    # FC layers with 128 neurons
    # Batchnorm
    # FC layers with 10 neurons


x = Conv2D(10, (5, 5))(inputs) # 5x5 convolutional layer with 10 output channels
x = MaxPooling2D(pool_size=(2, 2))(x) # maxpool 2
x = Conv2D(20, kernel_size=(3, 3))(x) # 3x3 convolution with 20 output channels
x = BatchNormalization()(x) # Batchnorm
x = Flatten()(x) # Flatten
x = Dropout(0.5)(x) # dropout = 0.5
x = Dense(128)(x) # FC layers with 128 neurons
x = BatchNormalization()(x) # Batchnorm
x = Dense(10)(x) # FC layers with 10 neurons

# Make sure there are 10 outputs!
x = Dense(10)(x)

model = tf.keras.Model(inputs=inputs, outputs=x)

#Select an appropriate keras optimizer and adjust its parameters: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# use Adam
opt = tf.keras.optimizers.Adam(
    learning_rate=0.00006,
    weight_decay=0.0001,
)

#Select an appropriate keras loss function. remember to set from_logits=True if your model does not pass your outputs
#through a softmax function: https://www.tensorflow.org/api_docs/python/tf/keras/losses
# sparse categorical cross entropy loss
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
)

model.compile(optimizer=opt,
              loss=loss_func,
              metrics=['accuracy'] #metrics tells the model what to print while we are training. We would like to see the accuracy
              )

In [ ]:
epochs = 20
batch_size = 32

In [ ]:
history = model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs,
                    batch_size=batch_size)

Epoch 1/20
38/38 [==============================] - 19s 253ms/step - loss: 0.0893 - accuracy: 0.9750 - val_loss: 3.7120 - val_accuracy: 0.6900
Epoch 2/20
38/38 [==============================] - 7s 181ms/step - loss: 0.0147 - accuracy: 0.9983 - val_loss: 0.4693 - val_accuracy: 0.8767
Epoch 3/20
38/38 [==============================] - 7s 185ms/step - loss: 0.0166 - accuracy: 0.9975 - val_loss: 0.0581 - val_accuracy: 0.9733
Epoch 4/20
38/38 [==============================] - 7s 181ms/step - loss: 0.0072 - accuracy: 0.9992 - val_loss: 0.0098 - val_accuracy: 1.0000
Epoch 5/20
38/38 [==============================] - 7s 185ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 6/20
38/38 [==============================] - 7s 180ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 7/20
38/38 [==============================] - 7s 186ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch

In [ ]:
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(val_accuracy)

10/10 [==============================] - 1s 51ms/step - loss: 3.9157e-04 - accuracy: 1.0000
1.0


In [ ]:
# put the collected data here as test data

In [ ]:
model.save('drive/My Drive/Extracurriculars/IEEE/PocketRacers/cnn_model.h5')

In [ ]:
model.load_weights('drive/My Drive/Extracurriculars/IEEE/PocketRacers/cnn_model.h5')